In [1]:
from SBM_SDE import *
from obs_and_flow_classes_and_functions_test import *
from get_CO2 import *
import seaborn as sns
import torch
from torch import nn
import torch.distributions as d
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import math
from tqdm import tqdm
import random
from torch.autograd import Function
import argparse
import os
import sys
from pathlib import Path
import shutil
import pandas as pd

In [2]:
torch.manual_seed(2)
devi = torch.device("".join(["cuda:",f'{cuda_id}']) if torch.cuda.is_available() else "cpu")

cuda_id = 1
dt = .2 #SDE discretization timestep.
t = 150 #Simulation run for T hours.
n = int(t / dt) 
t_span = np.linspace(0, t, n + 1)
t_span_tensor = torch.reshape(torch.Tensor(t_span), [1, n + 1, 1]) #T_span needs to be converted to tensor object. Additionally, facilitates conversion of I_S and I_D to tensor objects.
l_r = 1e-3
niter = 1500
piter = 100
batch_size = 2 #Number of sets of observation outputs to sample per set of parameters.
state_dim_SCON = 3 #Not including CO2 in STATE_DIM, because CO2 is an observation.
state_dim_SAWB = 4 #Not including CO2 in STATE_DIM, because CO2 is an observation.

In [3]:
obs_times, obs_means_CON, obs_error_CON = csv_to_obs_df('CON_synthetic_sol_df.csv', state_dim_SCON + 1, t, 0.1) #Need to +1 because data has CO2 observations on top of other state observations.
print(obs_means_CON)
print(obs_error_CON)

tensor([[4.5660e+01, 4.5647e+01, 4.5650e+01, 4.5643e+01, 4.5638e+01, 4.5640e+01,
         4.5626e+01, 4.5630e+01, 4.5618e+01, 4.5617e+01, 4.5615e+01, 4.5605e+01,
         4.5609e+01, 4.5595e+01, 4.5598e+01, 4.5591e+01],
        [7.1469e-02, 7.3700e-02, 7.3289e-02, 7.4423e-02, 7.5230e-02, 7.4875e-02,
         7.6935e-02, 7.6316e-02, 7.8034e-02, 7.7924e-02, 7.8087e-02, 7.9339e-02,
         7.8534e-02, 8.0440e-02, 7.9792e-02, 8.0558e-02],
        [7.1469e-01, 7.1372e-01, 7.1409e-01, 7.1363e-01, 7.1332e-01, 7.1362e-01,
         7.1266e-01, 7.1317e-01, 7.1240e-01, 7.1253e-01, 7.1251e-01, 7.1189e-01,
         7.1246e-01, 7.1149e-01, 7.1193e-01, 7.1158e-01],
        [1.1000e-03, 1.8381e-03, 4.6086e-04, 3.0443e-03, 4.7108e-04, 1.8966e-03,
         1.1670e-03, 7.0596e-04, 2.7906e-03, 4.2941e-04, 2.8298e-03, 7.3398e-04,
         1.2239e-03, 2.0338e-03, 5.1562e-04, 3.3495e-03]])
tensor([[4.5624e+00, 7.6809e-03, 7.1285e-02, 1.5369e-04]])


In [4]:
obs_times, obs_means_AWB, obs_error_AWB = csv_to_obs_df('AWB_synthetic_sol_df.csv', state_dim_SAWB + 1, t, 0.1) #Need to +1 because data has CO2 observations on top of other state observations.
print(obs_means_AWB)
print(obs_error_AWB)

tensor([[5.3602e+01, 5.3588e+01, 5.3591e+01, 5.3584e+01, 5.3578e+01, 5.3579e+01,
         5.3564e+01, 5.3569e+01, 5.3560e+01, 5.3559e+01, 5.3558e+01, 5.3545e+01,
         5.3551e+01, 5.3536e+01, 5.3538e+01, 5.3531e+01],
        [1.1421e-01, 1.1832e-01, 1.1810e-01, 1.1999e-01, 1.2125e-01, 1.2086e-01,
         1.2401e-01, 1.2294e-01, 1.2498e-01, 1.2491e-01, 1.2482e-01, 1.2692e-01,
         1.2544e-01, 1.2826e-01, 1.2713e-01, 1.2800e-01],
        [6.7073e-01, 6.7239e-01, 6.7187e-01, 6.7274e-01, 6.7348e-01, 6.7354e-01,
         6.7549e-01, 6.7470e-01, 6.7601e-01, 6.7623e-01, 6.7639e-01, 6.7818e-01,
         6.7750e-01, 6.7964e-01, 6.7946e-01, 6.8049e-01],
        [1.3415e-02, 1.3415e-02, 1.3415e-02, 1.3415e-02, 1.3415e-02, 1.3416e-02,
         1.3416e-02, 1.3416e-02, 1.3417e-02, 1.3417e-02, 1.3418e-02, 1.3419e-02,
         1.3419e-02, 1.3420e-02, 1.3421e-02, 1.3422e-02],
        [1.1000e-03, 1.6757e-03, 5.7473e-04, 2.4797e-03, 5.9680e-04, 1.7478e-03,
         1.2328e-03, 8.2910e-04, 2.4061

In [5]:
obs_times, obs_means_AWB_ECA, obs_error_AWB_ECA = csv_to_obs_df('AWB_ECA_synthetic_sol_df.csv', state_dim_SAWB + 1, t, 0.1) #Need to +1 because data has CO2 observations on top of other state observations.
print(obs_means_AWB_ECA)
print(obs_error_AWB_ECA)

tensor([[5.3606e+01, 5.3592e+01, 5.3594e+01, 5.3586e+01, 5.3579e+01, 5.3581e+01,
         5.3567e+01, 5.3572e+01, 5.3559e+01, 5.3558e+01, 5.3555e+01, 5.3545e+01,
         5.3550e+01, 5.3533e+01, 5.3538e+01, 5.3528e+01],
        [1.9081e-01, 1.9506e-01, 1.9527e-01, 1.9779e-01, 1.9962e-01, 1.9968e-01,
         2.0286e-01, 2.0196e-01, 2.0493e-01, 2.0501e-01, 2.0569e-01, 2.0746e-01,
         2.0642e-01, 2.0993e-01, 2.0873e-01, 2.1039e-01],
        [6.7073e-01, 6.7236e-01, 6.7196e-01, 6.7284e-01, 6.7358e-01, 6.7335e-01,
         6.7505e-01, 6.7448e-01, 6.7616e-01, 6.7630e-01, 6.7668e-01, 6.7806e-01,
         6.7749e-01, 6.7978e-01, 6.7921e-01, 6.8067e-01],
        [1.3415e-02, 1.3415e-02, 1.3415e-02, 1.3415e-02, 1.3415e-02, 1.3416e-02,
         1.3416e-02, 1.3416e-02, 1.3417e-02, 1.3417e-02, 1.3418e-02, 1.3418e-02,
         1.3419e-02, 1.3420e-02, 1.3421e-02, 1.3422e-02],
        [1.1000e-03, 1.6543e-03, 5.6910e-04, 2.4475e-03, 5.8824e-04, 1.7276e-03,
         1.2063e-03, 8.1481e-04, 2.3599

In [6]:
temp_ref = 283

#System parameters from deterministic CON model
u_M = 0.002
a_SD = 0.33
a_DS = 0.33
a_M = 0.33
a_MSC = 0.5
k_S_ref = 0.000025
k_D_ref = 0.005
k_M_ref = 0.0002
Ea_S = 75
Ea_D = 50
Ea_M = 50

#SCON diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
s_SOC = 0.1
s_DOC = 0.1
s_MBC = 0.1

SCON_C_params_dict = {'u_M': u_M, 'a_SD': a_SD, 'a_DS': a_DS, 'a_M': a_M, 'a_MSC': a_MSC, 'k_S_ref': k_S_ref, 'k_D_ref': k_D_ref, 'k_M_ref': k_M_ref, 'Ea_S': Ea_S, 'Ea_D': Ea_D, 'Ea_M': Ea_M, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC}
SCON_SS_params_dict = {'u_M': u_M, 'a_SD': a_SD, 'a_DS': a_DS, 'a_M': a_M, 'a_MSC': a_MSC, 'k_S_ref': k_S_ref, 'k_D_ref': k_D_ref, 'k_M_ref': k_M_ref, 'Ea_S': Ea_S, 'Ea_D': Ea_D, 'Ea_M': Ea_M, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC}

In [7]:
#System parameters from deterministic AWB model
u_Q_ref = 0.2
Q = 0.002
a_MSA = 0.5
K_D = 200
K_U = 1
V_D_ref = 0.4
V_U_ref = 0.02
Ea_V_D = 75
Ea_V_U = 50
r_M = 0.0004
r_E = 0.00001
r_L = 0.0005

#SAWB diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
c_EEC = 0.001
s_SOC = 0.1
s_DOC = 0.1
s_MBC = 0.1
s_EEC = 0.1

SAWB_C_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_D': K_D, 'K_U': K_U, 'V_D_ref': V_D_ref, 'V_U_ref': V_U_ref, 'Ea_V_D': Ea_V_D, 'Ea_V_U': Ea_V_U, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC, 'c_EEC': c_EEC}
SAWB_SS_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_D': K_D, 'K_U': K_U, 'V_D_ref': V_D_ref, 'V_U_ref': V_U_ref, 'Ea_V_D': Ea_V_D, 'Ea_V_U': Ea_V_U, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC, 's_EEC': s_EEC}

In [8]:
#System parameters from deterministic model
u_Q_ref = 0.2
Q = 0.002
a_MSA = 0.5
K_DE = 200
K_UE = 1
V_DE_ref = 0.4
V_UE_ref = 0.02
Ea_V_DE = 75
Ea_V_UE = 50
r_M = 0.0004
r_E = 0.00001
r_L = 0.0005

#Diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
c_EEC = 0.001
s_SOC = 0.1
s_DOC = 0.1
s_MBC = 0.1
s_EEC = 0.1

SAWB_ECA_C_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_DE': K_DE, 'K_UE': K_UE, 'V_DE_ref': V_DE_ref, 'V_UE_ref': V_UE_ref, 'Ea_V_DE': Ea_V_DE, 'Ea_V_UE': Ea_V_UE, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC, 'c_EEC': c_EEC}
SAWB_ECA_SS_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_DE': K_DE, 'K_UE': K_UE, 'V_DE_ref': V_DE_ref, 'V_UE_ref': V_UE_ref, 'Ea_V_DE': Ea_V_DE, 'Ea_V_UE': Ea_V_UE, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC, 's_EEC': s_EEC}

In [9]:
#Obtain SOC and DOC pool litter inputs for all SBMs.
i_s_tensor = 0.001 + 0.0005 * torch.sin((2 * np.pi / (24 * 365)) * t_span_tensor) #Exogenous SOC input function
i_d_tensor = 0.0001 + 0.00005 * torch.sin((2 * np.pi / (24 * 365)) * t_span_tensor) #Exogenous DOC input function

In [10]:
def neg_log_lik(C_PATH, T_SPAN_TENSOR, DT, I_S_TENSOR, I_D_TENSOR, DRIFT_DIFFUSION, PARAMS_DICT, TEMP_GEN, TEMP_REF):
    drift, diffusion_sqrt = DRIFT_DIFFUSION(C_PATH[:, :-1, :], T_SPAN_TENSOR[:, :-1, :], I_S_TENSOR[:, :-1, :], I_D_TENSOR[:, :-1, :], PARAMS_DICT, TEMP_GEN, TEMP_REF)
    euler_maruyama_state_sample_object = d.multivariate_normal.MultivariateNormal(loc = C_PATH[:, :-1, :] + drift * DT, scale_tril = diffusion_sqrt * math.sqrt(DT))
    return -euler_maruyama_state_sample_object.log_prob(C_PATH[:, 1:, :]).sum(-1)

In [13]:
def train(DEVICE, L_R, NITER, PRETRAIN_ITER, BATCH_SIZE, SDEFLOW, ObsModelCO2, csv_to_obs_df, DATA_CSV, OBS_ERROR_SCALE, STATE_DIM, T, DT, N, T_SPAN_TENSOR, I_S_TENSOR, I_D_TENSOR, DRIFT_DIFFUSION, PARAMS_DICT, TEMP_GEN, TEMP_REF, ANALYTICAL_STEADY_STATE_INIT, GET_CO2):
    obs_times, obs_means, obs_error = csv_to_obs_df(DATA_CSV, STATE_DIM + 1, T, OBS_ERROR_SCALE) #Need to +1 because data has CO2 observations on top of other state observations. 
    obs_model_CO2 = ObsModelCO2(DEVICE, obs_times, DT, obs_means, obs_error, GET_CO2)
    net = SDEFLOW(DEVICE, BATCH_SIZE, obs_model_CO2, STATE_DIM, T, DT, N).to(DEVICE)
    optimizer = optim.Adam(net.parameters(), lr = L_R) 
    if PRETRAIN_ITER >= NITER:
        raise Exception("PRETRAIN_ITER must be < NITER.")
    best_loss_norm = 1e10
    best_loss_ELBO = 1e20
    norm_losses = [best_loss_norm] * 10
    ELBO_losses = [best_loss_ELBO] * 10
    C0 = ANALYTICAL_STEADY_STATE_INIT(I_S_TENSOR[0, 0, 0].item(), I_D_TENSOR[0, 0, 0].item(), PARAMS_DICT) #Calculate deterministic initial conditions.
    C0 = C0[(None,) * 2].repeat(BATCH_SIZE, 1, 1).to(DEVICE) #Assign initial conditions to C_PATH.
    with tqdm(total = NITER, desc = f'Train Diffusion', position = -1) as tq:
        for iter in range(NITER):
            net.train()
            optimizer.zero_grad()
            C_PATH, log_prob = net() #Obtain paths with solutions at times after t0.
            C_PATH = torch.cat([C0, C_PATH], 1) #Append deterministic CON initial conditions conditional on parameter values to C path. 
            CO2 = GET_CO2(C_PATH, T_SPAN_TENSOR, PARAMS_DICT, TEMP_GEN, TEMP_REF)
            x_with_CO2 = torch.cat([C_PATH, CO2], -1)
            if iter <= PRETRAIN_ITER:
                #l1_norm_element = x_with_CO2 - torch.mean(obs_model_CO2.mu, -1)
                #l1_norm = torch.sum(torch.abs(l1_norm_element)).mean()
                #best_loss_norm = l1_norm if l1_norm < best_loss_norm else best_loss_norm
                #norm_losses.append(l1_norm.item())
                l2_norm_element = x_with_CO2 - torch.mean(obs_model_CO2.mu, -1)
                l2_norm = torch.sqrt(torch.sum(torch.square(l2_norm_element))).mean()
                best_loss_norm = l2_norm if l2_norm < best_loss_norm else best_loss_norm
                norm_losses.append(l2_norm.item())
                if len(norm_losses) > 10:
                    norm_losses.pop(0)
                if iter % 10 == 0:
                    print(f"Moving average norm loss at {iter} iterations is: {sum(norm_losses) / len(norm_losses)}. Best norm loss value is: {best_loss_norm}.")
                    print('\nx with CO2 means across time =', x_with_CO2.mean(-2))
                    print('\nx with CO2 =', x_with_CO2)
                #l1_norm.backward()
                l2_norm.backward()
            else:
                log_lik = neg_log_lik(C_PATH, T_SPAN_TENSOR.to(DEVICE), dt, I_S_TENSOR.to(DEVICE), I_D_TENSOR.to(DEVICE), DRIFT_DIFFUSION, PARAMS_DICT, TEMP_GEN, TEMP_REF)
                ELBO = log_prob.mean() + log_lik.mean() - obs_model_CO2(C_PATH) #obs_model_CO2(x) is obs log likelihood.
                best_loss_ELBO = ELBO if ELBO < best_loss_ELBO else best_loss_ELBO
                ELBO_losses.append(ELBO.item())
                if len(ELBO_losses) > 10:
                    ELBO_losses.pop(0)
                if iter % 10 == 0:
                    print(f"Moving average ELBO loss at {iter} iterations is: {sum(ELBO_losses) / len(ELBO_losses)}. Best ELBO loss value is: {best_loss_ELBO}.")
                    print('\nx means across time =', C_PATH.mean(-2))
                    print('\nx =', C_PATH)
                ELBO.backward()
            torch.nn.utils.clip_grad_norm_(net.parameters(), 3.0)
            optimizer.step()
            if iter % 100000 == 0 and iter > 0:
                optimizer.param_groups[0]['lr'] *= 0.1
            tq.update()

In [14]:
train(devi, l_r, niter, piter, batch_size, SDEFlow, ObsModelCO2, csv_to_obs_df, 'CON_synthetic_sol_df.csv', 0.1, state_dim_SCON, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SCON_C, SCON_C_params_dict, temp_gen, temp_ref, analytical_steady_state_init_CON, get_CO2_CON)


Train Diffusion:   0%|          | 0/1500 [00:00<?, ?it/s]

Moving average norm loss at 0 iterations is: 9000000173.522863. Best norm loss value is: 1735.2286376953125.

x with CO2 means across time = tensor([[0.8957, 1.0613, 0.8772, 0.0044],
        [0.8913, 1.0521, 0.9019, 0.0044]], grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [8.8302e-01, 6.4168e-01, 7.6020e-01, 2.3575e-03],
         [1.3240e+00, 1.3269e+00, 7.3454e-01, 4.9390e-03],
         ...,
         [9.3857e-01, 1.1499e+00, 2.6612e-01, 8.6721e-03],
         [7.2831e-01, 1.7995e+00, 1.1586e+00, 1.3792e-02],
         [8.2507e-01, 4.9178e-01, 1.4711e+00, 4.1523e-03]],

        [[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [8.8257e-01, 5.2927e-01, 1.1691e+00, 2.0228e-03],
         [1.1050e+00, 5.9500e-01, 1.1285e+00, 2.3402e-03],
         ...,
         [7.1644e-01, 7.2275e-01, 1.2659e+00, 5.7842e-03],
         [6.6725e-01, 5.8953e-01, 2.1581e-01, 4.4933e-03],
         [1.2247e+00, 1.4562e+00, 5.7891e-01, 1.1101e-02


Train Diffusion:   1%|          | 10/1500 [00:07<15:48,  1.57it/s]

Moving average norm loss at 10 iterations is: 1626.6730712890626. Best norm loss value is: 1563.1077880859375.

x with CO2 means across time = tensor([[5.3016e+00, 4.4716e-01, 3.8981e-01, 1.9784e-03],
        [5.9151e+00, 4.5250e-01, 3.9075e-01, 2.0138e-03]],
       grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [5.6950e-01, 1.5854e+00, 4.4895e-01, 5.5968e-03],
         [3.5474e+00, 7.8282e-01, 4.6120e-01, 2.9700e-03],
         ...,
         [2.7678e+00, 3.6618e-01, 3.9722e-01, 2.9910e-03],
         [8.1588e+00, 2.1772e-01, 4.2427e-01, 2.2016e-03],
         [9.1296e+00, 4.2573e-01, 3.4873e-01, 3.7870e-03]],

        [[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [1.1798e+00, 7.2768e+00, 3.8376e-01, 2.5430e-02],
         [9.5813e-01, 7.5177e-01, 5.6184e-01, 2.8234e-03],
         ...,
         [1.3939e+01, 4.5921e-01, 3.5485e-01, 4.2873e-03],
         [2.4127e+00, 4.8732e-01, 2.3326e-01, 3.8340e-03],
         [2.030


Train Diffusion:   1%|          | 15/1500 [00:10<17:59,  1.38it/s]


KeyboardInterrupt: 